# Mixed effect model analysis

## Load data and packages

In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import statsmodels.formula.api as smf

In [29]:
df_longer_weight = pd.read_excel("./data/df_long_format_for_analysis.xlsx",index_col=0)
df_longer_weight

,ID_Experiment,Mouse_ID,Date,Infection,Group,exp,survival_original,t_origin,Time,weight
0,ID_001,TRO-05432,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
1,ID_001,TRO-05433,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
2,ID_001,TRO-05434,2014-06-05,C. albicans,1A,1,1,9.0,Tinfection,100.0
3,ID_001,TRO-05435,2014-06-05,C. albicans,1A,1,1,6.0,Tinfection,100.0
4,ID_001,TRO-05456,2014-06-05,C. albicans,1A,1,1,7.0,Tinfection,100.0
...,...,...,...,...,...,...,...,...,...,...
32993,ID_096,TRO-028337,2023-03-03,S. pneumoniae,3,3,1,5.0,T13,NaN
32994,ID_096,TRO-028338,2023-03-03,S. pneumoniae,3,3,1,4.0,T13,NaN
32995,ID_096,TRO-028339,2023-03-03,S. pneumoniae,3,3,1,6.0,T13,NaN
32996,ID_096,TRO-028342,2023-03-03,S. pneumoniae,3,3,0,8.0,T13,NaN


## Mixed effect model
### Function

In [19]:
def Mixed_Effects_Models(df,chosen_infection = 'S. pneumoniae',time_to_exclude = 8):
    df_infection = df[df['Infection'] == chosen_infection]
    df_infection = df_infection[~df_infection['Time'].isin([f"T{n}" for n in range(time_to_exclude,15,1)])]#remove unused data
    
    time_point = df_infection['Time'].unique()
    weight_point_to_integer = dict(zip(time_point,[n for n in range(len(time_point))]))
    
    df_infection['Time'] = df_infection['Time'].replace(weight_point_to_integer)
    model = smf.mixedlm("weight ~ Time",df_infection,groups=df_infection['survival_original'],missing="drop").fit()
    return model.summary()

S. pneumoniae 3 days

In [28]:
# model with 3 days
Mixed_Effects_Models(df_longer_weight,"S. pneumoniae",3)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 2118    Method:             REML      
No. Groups:       2       Scale:              24.4583   
Min. group size:  981     Log-Likelihood:     -6394.4628
Max. group size:  1137    Converged:          Yes       
Mean group size:  1059.0                                
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept   99.816    0.922 108.310 0.000 98.010 101.622
Time        -2.233    0.131 -17.003 0.000 -2.491  -1.976
Group Var    1.642    0.478                             
========================================================

"""

In [27]:
# full model
Mixed_Effects_Models(df_longer_weight,"S. pneumoniae",14)

<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
=========================================================
Model:            MixedLM Dependent Variable: weight     
No. Observations: 4682    Method:             REML       
No. Groups:       2       Scale:              52.6772    
Min. group size:  1836    Log-Likelihood:     -15929.4311
Max. group size:  2846    Converged:          Yes        
Mean group size:  2341.0                                 
---------------------------------------------------------
              Coef.  Std.Err.   z    P>|z| [0.025  0.975]
---------------------------------------------------------
Intercept     97.369    2.625 37.097 0.000 92.225 102.513
Time          -0.229    0.040 -5.738 0.000 -0.308  -0.151
Group Var     13.721    2.729                            
=========================================================

"""

L. monocytogenes

In [21]:
# model with 3 days
Mixed_Effects_Models(df_longer_weight,"Listeria",3)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 3104    Method:             REML      
No. Groups:       2       Scale:              13.6527   
Min. group size:  1442    Log-Likelihood:     -8466.5944
Max. group size:  1662    Converged:          Yes       
Mean group size:  1552.0                                
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept  100.706    1.333  75.551 0.000 98.094 103.319
Time        -4.890    0.081 -60.147 0.000 -5.049  -4.731
Group Var    3.532    1.347                             
========================================================

"""

In [26]:
# full model
Mixed_Effects_Models(df_longer_weight,"Listeria",14)

<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
=========================================================
Model:            MixedLM Dependent Variable: weight     
No. Observations: 5311    Method:             REML       
No. Groups:       2       Scale:              53.6760    
Min. group size:  1913    Log-Likelihood:     -18118.5761
Max. group size:  3398    Converged:          Yes        
Mean group size:  2655.5                                 
---------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025  0.975]
---------------------------------------------------------
Intercept    96.000    2.963  32.402 0.000 90.193 101.807
Time         -1.249    0.049 -25.741 0.000 -1.344  -1.154
Group Var    17.510    3.378                             
=========================================================

"""

C. albicans

In [22]:
# model with 5 days
Mixed_Effects_Models(df_longer_weight,"C. albicans",5)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 1199    Method:             REML      
No. Groups:       2       Scale:              38.4846   
Min. group size:  379     Log-Likelihood:     -3892.7082
Max. group size:  820     Converged:          Yes       
Mean group size:  599.5                                 
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept   97.468    1.627  59.907 0.000 94.279 100.657
Time        -3.865    0.127 -30.525 0.000 -4.113  -3.617
Group Var    5.102    1.181                             
========================================================

"""

In [25]:
# full model
Mixed_Effects_Models(df_longer_weight,"C. albicans",14)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 2337    Method:             REML      
No. Groups:       2       Scale:              82.3700   
Min. group size:  605     Log-Likelihood:     -8475.4109
Max. group size:  1732    Converged:          Yes       
Mean group size:  1168.5                                
--------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025 0.975]
--------------------------------------------------------
Intercept    90.420    4.521  20.001 0.000 81.560 99.280
Time         -0.795    0.052 -15.299 0.000 -0.897 -0.694
Group Var    40.668    6.406                            
========================================================

"""

H1N1

In [23]:
# model with 5 days
Mixed_Effects_Models(df_longer_weight,"H1N1",5)

<class 'statsmodels.iolib.summary2.Summary'>
"""
         Mixed Linear Model Regression Results
========================================================
Model:            MixedLM Dependent Variable: weight    
No. Observations: 1666    Method:             REML      
No. Groups:       2       Scale:              35.8096   
Min. group size:  651     Log-Likelihood:     -5348.6435
Max. group size:  1015    Converged:          Yes       
Mean group size:  833.0                                 
--------------------------------------------------------
            Coef.  Std.Err.    z    P>|z| [0.025  0.975]
--------------------------------------------------------
Intercept  100.074    2.232  44.837 0.000 95.699 104.448
Time        -2.995    0.104 -28.759 0.000 -3.199  -2.790
Group Var    9.833    2.335                             
========================================================

"""

In [24]:
# full model
Mixed_Effects_Models(df_longer_weight,"H1N1",14)

<class 'statsmodels.iolib.summary2.Summary'>
"""
          Mixed Linear Model Regression Results
=========================================================
Model:            MixedLM Dependent Variable: weight     
No. Observations: 2872    Method:             REML       
No. Groups:       2       Scale:              83.7580    
Min. group size:  891     Log-Likelihood:     -10438.6065
Max. group size:  1981    Converged:          Yes        
Mean group size:  1436.0                                 
---------------------------------------------------------
             Coef.  Std.Err.    z    P>|z| [0.025  0.975]
---------------------------------------------------------
Intercept    96.291    4.220  22.818 0.000 88.020 104.562
Time         -1.453    0.058 -25.060 0.000 -1.566  -1.339
Group Var    35.446    5.446                             
=========================================================

"""